# first attempt - simple reverse prediction

In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
gen = 2
name_df = f'{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [3]:
new_columns = [f'Col_{i}' for i in range(1, gen*SIZE*SIZE + 1)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [4]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 29760
reverse df sort: 29760


In [5]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                measure_error(y_test, y_test_pred, 'test')],
                                axis=1)

print(dt.tree_.node_count, dt.tree_.max_depth)
train_test_full_error

len x train:  26784
len x test:  2976
len y train:  26784
len y test:  2976
18129 25


,train,test
accuracy,0.969945,0.629704
precision,0.989717,0.460565
recall,0.922861,0.463271
f1,0.955121,0.461914


In [6]:
reverse_df_sort.iloc[21540].values

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0])

In [7]:
features.iloc[21540].values

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0])

In [8]:
target.iloc[21540]

0

In [9]:
depth_to_check = [4,5,6,7,8,9,10,15,20,30,50,100]

for i in depth_to_check:
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, i, 42)
    
    name_file = f"decision_tree_reverse_df_model_{str(i)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    
    print(node_count, max_depth)
    print(train_test_full_error)

31 4
              train      test
accuracy   0.681750  0.665659
precision  0.562894  0.519637
recall     0.365438  0.336925
f1         0.443167  0.408794
63 5
              train      test
accuracy   0.686828  0.668347
precision  0.573617  0.525680
recall     0.375242  0.340842
f1         0.453693  0.413547
127 6
              train      test
accuracy   0.688508  0.665323
precision  0.575689  0.518409
recall     0.384723  0.344760
f1         0.461221  0.414118
255 7
              train      test
accuracy   0.694407  0.677755
precision  0.606567  0.556159
recall     0.336350  0.300686
f1         0.432740  0.390337
511 8
              train      test
accuracy   0.703480  0.671035
precision  0.624350  0.534768
recall     0.362422  0.316357
f1         0.458623  0.397538
1003 9
              train      test
accuracy   0.715912  0.668683
precision  0.652562  0.529915
recall     0.385477  0.303624
f1         0.484660  0.386052
1927 10
              train      test
accuracy   0.737418  0.6612

In [10]:
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
for i in range(SIZE*SIZE): # to any pixel in the expected board
    name_col = 'Col_' + str(i+amount_features + 1)
    target = reverse_df_sort[name_col]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=613)
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, 20, 42)
    name_file = f"decision_tree_reverse_df_model_pixel_{str(i+1)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    print(i)
    print(node_count, max_depth)
    print(train_test_full_error)

0
17277 20
              train      test
accuracy   0.963187  0.626344
precision  0.988709  0.447130
recall     0.904260  0.440914
f1         0.944601  0.444000
1
16933 20
              train      test
accuracy   0.962403  0.630376
precision  0.983972  0.438247
recall     0.905531  0.450820
f1         0.943123  0.444444
2
17063 20
              train      test
accuracy   0.963112  0.626680
precision  0.983786  0.455897
recall     0.908542  0.450098
f1         0.944668  0.452979
3
16913 20
              train      test
accuracy   0.963747  0.617608
precision  0.986342  0.432049
recall     0.905721  0.424303
f1         0.944314  0.428141
4
17091 20
              train      test
accuracy   0.964307  0.632392
precision  0.986440  0.449438
recall     0.908656  0.442211
f1         0.945952  0.445795
5
17037 20
              train      test
accuracy   0.962478  0.645497
precision  0.983993  0.477551
recall     0.906566  0.462908
f1         0.943694  0.470116
6
17095 20
              train    